In [ ]:
import torch
from transformers import AutoTokenizer, RobertaModel
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from datasets import load_dataset
from utils import *

# Cargar el conjunto de datos y obtener el mapeo de emociones
dataset_name = "RikoteMaster/Emotion_Recognition_4_llama2_chat"
ds = load_dataset(dataset_name, split="train")
unique_emotions = set(ds['Emotion'])
one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(np.array(list(unique_emotions)).reshape(-1, 1))
emotion_mapping = one_hot_encoder.categories_[0]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the path to the trained model
classifier_state_dict_path = "/workspace/code/MODELS/RoBERTa_entrenado_base_-12/classifier_state_dict.pt"

# Load the trained model
classifier_state_dict = torch.load(classifier_state_dict_path)
base_model = RobertaModel.from_pretrained("roberta-base")
model = CustomRoBERTa(base_model, num_classes=len(unique_emotions))
model.classifier.load_state_dict(classifier_state_dict)
compiled_model = torch.compile(model).to(device)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Bucle para procesar inputs del usuario
while True:
    # Solicitar texto de entrada al usuario
    input_text = input("Introduce un texto para analizar la emoción (escribe 'salir' para terminar): ")
    
    # Comprobar si el usuario desea salir
    if input_text.lower() == 'salir':
        break

    # Tokenize y codifica el texto de entrada
    encoded_input = tokenizer.encode_plus(
        input_text, 
        add_special_tokens=True,
        padding='max_length', 
        truncation=True,
        max_length=340, 
        return_attention_mask=True,
        return_tensors='pt'
    )

    encoded_input = encoded_input.to(device)

    # Pasar el texto codificado a través del modelo
    model.eval()
    with torch.no_grad():
        outputs = model(encoded_input['input_ids'].to(device), attention_mask=encoded_input['attention_mask'].to(device))

    # Obtener el índice de la emoción predicha
    predicted_emotion_index = np.argmax(outputs.logits.detach().cpu().numpy(), axis=1)[0]

    # Traducir el índice predicho a una etiqueta legible
    predicted_emotion_label = emotion_mapping[predicted_emotion_index]
    print(f"La emoción predicha para el texto '{input_text}' es: {predicted_emotion_label}")


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Introduce un texto para analizar la emoción (escribe 'salir' para terminar): I hate when you do that
La emoción predicha para el texto 'I hate when you do that' es: anger
Introduce un texto para analizar la emoción (escribe 'salir' para terminar): I feel happy when we won the lottery
La emoción predicha para el texto 'I feel happy when we won the lottery' es: joy


The predicted emotion for the text 'I hate when you do that but it feels horrible being in love with you' is: [5]
